<a href="https://colab.research.google.com/github/jon-chun/sentimenttime/blob/main/web_bs4_webscraping_20210818.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webscrape Example

* https://github.com/julia-git/webscraping_ny_mta

In [77]:
# Connect to Google gDrive

# Flag to indicate first run through code 
flag_first_run = True

from google.colab import drive, files
drive.mount('/gdrive')
%cd /gdrive/MyDrive/

Mounted at /gdrive
/gdrive/MyDrive


In [90]:
gdrive_subdir = "./research/2021/sa_book_code/books_sa/books_raw"

In [92]:
%cd $gdrive_subdir

/gdrive/My Drive/research/2021/sa_book_code/books_sa/books_raw


In [93]:
!pwd

/gdrive/My Drive/research/2021/sa_book_code/books_sa/books_raw


In [94]:
!ls

In [ ]:
flag_first_run = True

CORPUS_FULL = 'Great Expectations by Charles Dickens (1861)'
CORPUS_SUBDIR = gdrive_subdir
corpus_filename = CORPUS_SUBDIR

# Change to working subdirectory
if flag_first_run == True:
  full_path_str = gdrive_subdir
  flag_first_run = False
else:
  full_path_str = f'/gdrive/MyDrive{gdrive_subdir[1:]}'

%cd $full_path_str

/gdrive/MyDrive/research/2021/sa_book_code/books_sa/cdickens_greatexpectations


In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

### Set the URL you want to webscrape from

In [2]:
url = 'http://www.glozman.com/textpages.html' # 'http://web.mta.info/developers/turnstile.html'

In [23]:
url_base = '/'.join(url.split('/')[:-1]) + '/'
print(f'url_base: {url_base}')

url_base: http://www.glozman.com/


### Connect to the URL

In [3]:
response = requests.get(url)

In [4]:
response #200 means it went through

<Response [200]>

### Parse HTML and save to BeautifulSoup object

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
soup

<body>
<b>PagesForPhone</b>
<ul>
<li><a href="TextPages/Clancy Tom - Patriot Games.txt">Clancy Tom - Patriot Games.txt</a></li>
<li><a href="TextPages/Clancy Tom - Red Storm Rising.txt">Clancy Tom - Red Storm Rising.txt</a></li>
<li><a href="TextPages/Clarke Arthur C - 3001 The Final Odissey.txt">Clarke Arthur C - 3001 The Final Odyssey.txt</a></li>
<li><a href="TextPages/Frank Herbert - Children of Dune.txt">Frank Herbert - Children of Dune.txt</a></li>
<li><a href="TextPages/Frank Herbert - Dune Messiah.txt">Frank Herbert - Dune Messiah.txt</a></li>
<li><a href="TextPages/Frank Herbert - Dune.txt">Frank Herbert - Dune.txt</a></li>
<li><a href="TextPages/Harry Potter 1 - Sorcerer's Stone.txt">Harry Potter 1 - Sorcerer's Stone.txt</a></li>
<li><a href="TextPages/Harry Potter 2 - Chamber of Secrets.txt">Harry Potter 2 - Chamber of Secrets.txt</a></li>
<li><a href="TextPages/Harry Potter 3 - The Prisoner Of Azkaban.txt">Harry Potter 3 - The Prisoner Of Azkaban.txt</a></li>
<li><a href="T

### To locate all  'a' tags

In [7]:
soup.findAll('a')

[<a href="TextPages/Clancy Tom - Patriot Games.txt">Clancy Tom - Patriot Games.txt</a>,
 <a href="TextPages/Clancy Tom - Red Storm Rising.txt">Clancy Tom - Red Storm Rising.txt</a>,
 <a href="TextPages/Clarke Arthur C - 3001 The Final Odissey.txt">Clarke Arthur C - 3001 The Final Odyssey.txt</a>,
 <a href="TextPages/Frank Herbert - Children of Dune.txt">Frank Herbert - Children of Dune.txt</a>,
 <a href="TextPages/Frank Herbert - Dune Messiah.txt">Frank Herbert - Dune Messiah.txt</a>,
 <a href="TextPages/Frank Herbert - Dune.txt">Frank Herbert - Dune.txt</a>,
 <a href="TextPages/Harry Potter 1 - Sorcerer's Stone.txt">Harry Potter 1 - Sorcerer's Stone.txt</a>,
 <a href="TextPages/Harry Potter 2 - Chamber of Secrets.txt">Harry Potter 2 - Chamber of Secrets.txt</a>,
 <a href="TextPages/Harry Potter 3 - The Prisoner Of Azkaban.txt">Harry Potter 3 - The Prisoner Of Azkaban.txt</a>,
 <a href="TextPages/Harry Potter 4 - The Goblet Of Fire.txt">Harry Potter 4 - The Goblet Of Fire.txt</a>,
 <a 

### Let's take a quick look at the very first data file, which starts on line 36

In [13]:
alink_ls = list(soup.findAll('a'))
print(alink_ls[:5])

[<a href="TextPages/Clancy Tom - Patriot Games.txt">Clancy Tom - Patriot Games.txt</a>, <a href="TextPages/Clancy Tom - Red Storm Rising.txt">Clancy Tom - Red Storm Rising.txt</a>, <a href="TextPages/Clarke Arthur C - 3001 The Final Odissey.txt">Clarke Arthur C - 3001 The Final Odyssey.txt</a>, <a href="TextPages/Frank Herbert - Children of Dune.txt">Frank Herbert - Children of Dune.txt</a>, <a href="TextPages/Frank Herbert - Dune Messiah.txt">Frank Herbert - Dune Messiah.txt</a>]


In [16]:
# one_a_tag = soup.findAll('a')[36]
one_a_tag = alink_ls[36]
print(one_a_tag)

<a href="TextPages/Salvatore, R.A. - Paths of Darkness 1 - The Silent Blade.txt">R.A. Salvatore - Paths of Darkness 1 - The Silent Blade.txt</a>


### We want to extract the actual link

In [17]:
link = one_a_tag['href']
link

'TextPages/Salvatore, R.A. - Paths of Darkness 1 - The Silent Blade.txt'

### The full download URL is 'http://web.mta.info/developers/' + link

## To download the whole data set, let's do a for loop through all a tags

In [28]:
import os

In [32]:
corpora_path = './corpora_data/'

In [33]:
if not os.path.isdir(corpora_path):
  os.makedirs(corpora_path)

In [34]:
!ls

corpora_data  data  sample_data


In [43]:
from pathlib import Path
import requests

def url_retrieve(url: str, outfile: Path):
  R = requests.get(url, allow_redirects=True)
  if R.status_code != 200:
      raise ConnectionError('could not download {}\nerror code: {}'.format(url, R.status_code))

  outfile.write_text(R.content)

In [55]:
import re

In [95]:
file_ct = 1
for one_a_tag in soup.findAll('a'): 
  # 'a' tags are for links
  # if line_count >= 36: #code for text files starts at line 36
  link = one_a_tag['href']
  # raw_fname = ''.join(link.split('\'))
  local_fname = ''.join([x.capitalize() for x in link.split()])
  local_fname = ''.join([x for x in local_fname.split('/')[1:]])
  # local_fname = str(local_file).capitalize()
  local_fname = re.sub(r'[^A-Za-z0-9. ]+', '', local_fname)


  download_url = (url_base + link).replace(" ", "%20")
  print(f'Downloading Link #{file_ct}:\n  {download_url}\n')
  
  R = requests.get(download_url, allow_redirects=True)
  if R.status_code != 200:
    raise ConnectionError('could not download {}\nerror code: {}'.format(url, R.status_code))
  else:
    print(f'  Saving remote file: {download_url}')
    print(f'       to local file: {local_fname}\n\n')

    with open(local_fname, 'w') as fp:
      fp.write(R.text)
      
  time.sleep(1)
  file_ct += 1

"""
  local_file = Path(link)
  url_retrieve(download_url, local_file)
  # urllib.request.urlretrieve(download_url, f'{corpora_path}{link}') # ,'./'+link[link.find('/turnstile_')+1:]) 
  time.sleep(1)
  line_count +=1
""";

  http://www.glozman.com/TextPages/Clancy%20Tom%20-%20Patriot%20Games.txt

  Saving remote file: http://www.glozman.com/TextPages/Clancy%20Tom%20-%20Patriot%20Games.txt
       to local file: clancyTomPatriotGames.txt


  http://www.glozman.com/TextPages/Clancy%20Tom%20-%20Red%20Storm%20Rising.txt

  Saving remote file: http://www.glozman.com/TextPages/Clancy%20Tom%20-%20Red%20Storm%20Rising.txt
       to local file: clancyTomRedStormRising.txt


  http://www.glozman.com/TextPages/Clarke%20Arthur%20C%20-%203001%20The%20Final%20Odissey.txt

  Saving remote file: http://www.glozman.com/TextPages/Clarke%20Arthur%20C%20-%203001%20The%20Final%20Odissey.txt
       to local file: clarkeArthurC3001TheFinalOdissey.txt


  http://www.glozman.com/TextPages/Frank%20Herbert%20-%20Children%20of%20Dune.txt

  Saving remote file: http://www.glozman.com/TextPages/Frank%20Herbert%20-%20Children%20of%20Dune.txt
       to local file: frankHerbertChildrenOfDune.txt


  http://www.glozman.com/TextPages/Frank

In [27]:
!ls -altr

total 16
drwxr-xr-x 4 root root 4096 Aug 13 13:34 .config
drwxr-xr-x 1 root root 4096 Aug 13 13:35 .
drwxr-xr-x 1 root root 4096 Aug 13 13:35 sample_data
drwxr-xr-x 1 root root 4096 Aug 20 07:29 ..
